In [20]:
from multiinstance.data.gaussian_dg import GaussianMixtureDataGenerator

import numpy as np

from easydict import EasyDict
import matplotlib.pyplot as plt

In [78]:
from sklearn.metrics import roc_auc_score

In [45]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Lambda, Activation, Concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as KB
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow import summary
import tensorflow.keras.backend as K
import tensorflow as tf

In [4]:
def generateParams(nClusters, nComps, dim,aucpn_range=[0.8,0.85]):
    posMeans,posCovs,negMeans,negCovs,dgs =[], [],[],[],[]
    for comp in range(nClusters):
        dg = GaussianMixtureDataGenerator(dim,nComps,aucpn_range,100,1000)
        pos,neg = dg.components_pos[0], dg.components_neg[0]
        r = np.zeros(dim)
        for _ in range(1):
            r += np.random.normal(loc=0,scale=10,size=dim)
        posMeans.append(pos.mean+r)
        posCovs.append(pos.cov)
        negMeans.append(neg.mean+r)
        negCovs.append(neg.cov)
        dgs.append(dg)
    val = EasyDict
    val.posMeans = posMeans
    val.posCovs = posCovs
    val.negMeans = negMeans
    val.negCovs = negCovs
    val.dgs = dgs
    return val

In [317]:
args = EasyDict()
args.nClusters = 1
args.nComps = 1
args.dim = 10
args.aucpn_range=[0.7,.8]
args.hdim = 4

In [318]:
data = generateParams(args.nClusters, args.nComps, args.dim, args.aucpn_range)

In [319]:
# rnge = np.arange(-5,15,.1)
# plt.plot(rnge,data.dgs[0].pn_posterior_cc(rnge))
# plt.plot(rnge,data.dgs[0].dens_pos(rnge),label="f1(x)")
# plt.plot(rnge,data.dgs[0].dens_neg(rnge),label="f0(x)")
# plt.legend()

In [320]:
x,y = data.dgs[0].pn_data(20000,.5)
y = y.astype(bool)

In [321]:
model = Sequential(name="model")
model.add(Dense(args.hdim, input_shape=(args.dim,), activation='relu',))
model.add(Dense(args.hdim, activation='relu'))
model.add(Dense(1, activation="sigmoid"))


In [322]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy())

In [323]:
# NMixup = 20000
# randindx1 = np.random.choice(np.arange(p.shape[0]),size=NMixup)
# randindx2 = np.random.choice(np.arange(p.shape[0]),size=NMixup)
# beta = np.random.beta(2,2,size=NMixup)
# m = x[y][randindx1] * beta + (1 - beta) * x[~y][randindx2]
# m = m.reshape((-1,1))
# beta = beta.reshape((-1,1))

# model.fit(np.concatenate((x,m)),np.concatenate((y,beta)),epochs=20)

model.fit(x,y,epochs=20)

Epoch 1/20
625/625 [==============================] - 1s 1ms/step - loss: 0.6783
Epoch 2/20
625/625 [==============================] - 1s 1ms/step - loss: 0.6251
Epoch 3/20
625/625 [==============================] - 1s 1ms/step - loss: 0.6129
Epoch 4/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5963
Epoch 5/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5929
Epoch 6/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5921
Epoch 7/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5869
Epoch 8/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5864
Epoch 9/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5818
Epoch 10/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5918
Epoch 11/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5888
Epoch 12/20
625/625 [==============================] - 1s 1ms/step - loss: 0.5917
Epoch 13/20
625/625 [====

In [324]:
# rnge = np.arange(-5,15,.1)
# plt.plot(rnge,data.dgs[0].pn_posterior_cc(rnge),label="p")
# plt.plot(rnge,data.dgs[0].dens_pos(rnge),label="f1(x)")
# plt.plot(rnge,data.dgs[0].dens_neg(rnge),label="f0(x)")
# plt.plot(rnge,model.predict(rnge.reshape((-1,1))),label="pHat")
# # plt.hist(x[y.astype(bool)],density=True,label="p")
# # plt.hist(x[~y.astype(bool)],density=True,label="n")
# plt.legend()

In [325]:
roc_auc_score(y,model.predict(x))

0.7556967149999999

In [326]:
roc_auc_score(y, data.dgs[0].pn_posterior_cc(x))

0.8004858799999999